In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [9]:
from langchain_community.document_loaders import PDFPlumberLoader

In [10]:
# pip install pdfplumber

In [11]:
loader = PDFPlumberLoader('./data/converted_data_with_metadata.pdf')

In [12]:
docs = loader.load()

In [67]:
len(docs)

21

In [68]:
docs

[Document(metadata={'source': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'file_path': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'page': 3, 'total_pages': 25, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13947', 'Producer': 'Hancom PDF 1.3.0.547', 'CreationDate': "D:20241105150426+09'00'", 'ModDate': "D:20241105150426+09'00'", 'PDFVersion': '1.4'}, page_content='1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석\nKEY Contents\nn 미국 민권위원회에 따르면 연방정부와 법 집행기관에서 얼굴인식 기술이 빠르게 도입되고\n있으나 이를 관리할 지침과 감독의 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회는 연방정부의 책임 있는 얼굴인식 기술 사용을 위해 운영 프로토콜 개발과\n실제 사용 상황의 얼굴인식 기술 평가 및 불평등 완화, 지역사회의 의견 수렴 등을 권고\n£연방정부의 얼굴인식 기술 도입에 대한 지침과 감독 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회(U.S. Commission on Civil Rights)가 2024년 9월 19일 연방정부의 얼굴인식\n기술 사용이 민권에 미치는 영향을 분석한 보고서를 발간\n∙ AI 기술의 일종인 얼굴인식 기술은 연방정부와 법 집행기관에서 빠르게 도입되고 있으며, 일례로\n법무부 연방수사국(FBI)은 범죄 수사 및 용의자 수색용 단서 확보를 위해 얼굴인식 기술을 가장 빈번히 사용\n∙ 그러나 얼굴인식 기술의 책임 있는 사용을 위한 연방 지침과 감독은 실제 활용 사례보다 뒤처졌으며,\n현재 연방정부의 얼굴인식 기술이나 여타 AI 기술 

In [69]:
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

In [70]:
docs

[Document(metadata={'source': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'file_path': './data/SPRi AI Brief_11월호_산업동향_F.pdf', 'page': 3, 'total_pages': 25, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13947', 'Producer': 'Hancom PDF 1.3.0.547', 'CreationDate': "D:20241105150426+09'00'", 'ModDate': "D:20241105150426+09'00'", 'PDFVersion': '1.4', 'filename': './data/SPRi AI Brief_11월호_산업동향_F.pdf'}, page_content='1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석\nKEY Contents\nn 미국 민권위원회에 따르면 연방정부와 법 집행기관에서 얼굴인식 기술이 빠르게 도입되고\n있으나 이를 관리할 지침과 감독의 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회는 연방정부의 책임 있는 얼굴인식 기술 사용을 위해 운영 프로토콜 개발과\n실제 사용 상황의 얼굴인식 기술 평가 및 불평등 완화, 지역사회의 의견 수렴 등을 권고\n£연방정부의 얼굴인식 기술 도입에 대한 지침과 감독 부재로 민권 문제를 초래할 위험 존재\nn 미국 민권위원회(U.S. Commission on Civil Rights)가 2024년 9월 19일 연방정부의 얼굴인식\n기술 사용이 민권에 미치는 영향을 분석한 보고서를 발간\n∙ AI 기술의 일종인 얼굴인식 기술은 연방정부와 법 집행기관에서 빠르게 도입되고 있으며, 일례로\n법무부 연방수사국(FBI)은 범죄 수사 및 용의자 수색용 단서 확보를 위해 얼굴인식 기술을 가장 빈번히 사용\n∙ 그러나 얼굴인식 기술의 책임 있는 사용을 위한 연방 지침과

In [44]:
# !pip install -q ragas==0.1.19

In [71]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [139]:
# 생성기
generator_llm = ChatOpenAI(model = 'gpt-4o')

In [140]:
# 평가를 할 수 있는 비평기
critic_llm = ChatOpenAI(model = 'gpt-4o')

In [141]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [142]:
# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [143]:
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model = 'gpt-4o'))
# 구문추출기 생성을 위한 모델 호출(RAGAS와의 호환을 위한 Wrapper)

In [144]:
# 구문추출기 : 문서 핵심 정보 식별 및 추출 역할
Keyphrase_extractor = KeyphraseExtractor(llm = langchain_llm)

In [145]:
# 임베딩 모델도 RAGAS와 호환을 위해 Wrapper 적용
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

In [146]:
# 저장소, docstore 만들기
docstore = InMemoryDocumentStore(
    splitter = splitter,
    embeddings = ragas_embeddings,
    extractor = Keyphrase_extractor
)

In [147]:
# 생성기 만들기
generator = TestsetGenerator.from_langchain(
    generator_llm, # 생성기
    critic_llm,  # 판별기
    ragas_embeddings, # 임베딩모델
    docstore=docstore # 문서저장소
)

In [148]:
# # 질문 유형 분포를 설정하는 딕셔너리입니다.
# # 각 질문 유형과 그에 해당하는 비율(0~1)을 정의합니다.
# distributions = {
#     "simple": 0.4,          # 단순한 질문 유형의 비율 (전체의 40%)
#     "reasoning": 0.2,       # 논리적 사고가 필요한 질문 유형의 비율 (전체의 20%)
#     "multi_context": 0.2,   # 여러 문맥을 고려해야 하는 질문 유형의 비율 (전체의 20%)
#     "conditional": 0.2      # 조건에 따라 답변이 달라지는 질문 유형의 비율 (전체의 20%)
# }

In [149]:
# 질문 유형 분포 결정
distributions = {simple : 0.4, reasoning : 0.2, multi_context : 0.2, conditional : 0.2}

In [151]:
testset = generator.generate_with_langchain_docs(
    documents = docs,
    test_size = 10,
    distributions = distributions,
    with_debugging_logs = True,
    # 로그 활성화
    raise_exceptions= False
    # 예외가 발생해도 계속 진행, 예외 발생 시 문서가 처리 되지 않거나 문서에 기록됨
)

embedding nodes:  16%|█▌        | 13/82 [00:29<02:38,  2.29s/it]Task exception was never retrieved
future: <Task finished name='Task-12249' coro=<as_completed.<locals>.sema_coro() done, defined at c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages\ragas\executor.py:32> exception=NotFoundError("Error code: 404 - {'error': {'message': 'The model `gpt-3.5` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}")>
Traceback (most recent call last):
  File "c:\Users\RMARKET\anaconda3\envs\langchain\Lib\asyncio\tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages\ragas\executor.py", line 34, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages\ragas\executor.py", line 61, in wrapped_callable_async
    raise e
  File "c:\Users\RMARKET\anacon

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-nS8RoafnzeRRZRKTRiJUJQQC on tokens per min (TPM): Limit 30000, Used 30000, Requested 936. Please try again in 1.872s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['해결 역량', '물리적 작업', '생성AI', '인간 근로자 대체 가능성', '디지털 기술']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['NeurIPS 2024', 'GenAI Summit Maroc 2024', 'AI Summit Seoul', 'Artificial Intelligence', 'Data Analysis']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Generative AI governance', 'Diverse stakeholders', 'AI accountability', 'International cooperation', 'AI safety research']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Data collection standardization', 'AI model development', 'Data privacy and human rights', 'EU AI law', 'High-risk AI systems']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['DeepMind CEO John Jumper', 'AI-based protein structure prediction', 'AlphaFold 2', 'Nobel Prize in Chemistry 2024', 'Protein folding problem']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI 거버넌스 프레임워크', 'G7 회원국', 'AI 관리 구조', '공공 부문 AI 도입', '데이터 저장과 공유 솔루션']}
[ragas.testset.extractor.DEBU

In [153]:
test_df = testset.to_pandas()
test_df

""


### 데이터 셋이 나오지 않았기 때문에 우선 csv 파일을 이용

In [154]:
import pandas as pd

df = pd.read_csv(r'C:\Users\RMARKET\workspace\5주차\data\ragas_dataset.csv')

In [155]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What specific aspects of data collection stand...,"['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n유로폴, 법 ...",The answer to given question is not present in...,simple,[{'source': 'data/SPRi AI Brief_11월호_산업동향_F.pd...,True
1,What factors are contributing to the recent in...,"['SPRi AI Brief |\n2024-11월호\nAI21 CEO, AI 에이전...",The recent increase in popularity of the Mamba...,simple,[{'source': 'data/SPRi AI Brief_11월호_산업동향_F.pd...,True
2,How does the MIT Industrial Performance Center...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\nMIT 산업성...,The MIT Industrial Performance Center assesses...,simple,[{'source': 'data/SPRi AI Brief_11월호_산업동향_F.pd...,True
3,What role does AI play in addressing global ch...,"['∙ (글로벌 도전과제 해결) 환경 문제, 경제 회복력, 사회복지 등 글로벌 도전...",AI helps in solving global challenges such as ...,simple,[{'source': 'data/SPRi AI Brief_11월호_산업동향_F.pd...,True
4,How do Transformers boost AI21's model efficie...,"['SPRi AI Brief |\n2024-11월호\nAI21 CEO, AI 에이전...",Transformers are used extensively in AI model ...,reasoning,[{'source': 'data/SPRi AI Brief_11월호_산업동향_F.pd...,True


In [156]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [157]:
df['contexts'][0]

"['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\\n유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간\\nKEY Contents\\nn 유로폴의 보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오\\n분석 등에 활용되어 법 집행 업무를 대폭 개선할 수 있는 잠재력 보유\\nn 그러나 AI 도입을 위해서는 기술적 과제 해결 및 다양한 윤리적·사회적 이슈 대응이\\n필요하며, EU AI 법에 부합하도록 기존 AI 시스템에 대한 평가와 수정도 필요\\n£유로폴, 법 집행에서 AI 기술의 윤리적이고 투명한 구현을 위한 고려사항 제시\\nn EU 사법기관 유로폴(Europol)이 2024년 9월 24일 법 집행에서 효과적 범죄 퇴치를 위한 AI의\\n활용 가능성을 탐색한 보고서를 발간\\n∙ 보고서는 법 집행에서 AI 기술을 윤리적이고 투명하게 구현하기 위한 지침 역할을 하며, AI의 이점과\\n과제를 함께 다룸으로써 법 집행에서 AI 사용 시 윤리적 고려 사항에 대한 인식 제고를 추구\\nn 보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오 분석, 생체인식\\n시스템 등에 활용되어 법 집행 업무를 대폭 개선할 수 있는 잠재력 보유\\n∙ 법 집행기관은 AI 기반 데이터 분석을 활용해 범죄 활동에 대한 탐지와 대응 능력을 강화하고, AI\\n도구로 구조화되지 않은 데이터를 신속히 분석해 비상 상황의 의사결정을 위한 통찰력 확보 가능\\n∙ 기계번역과 같은 AI 기반 도구는 여러 국가가 참여하는 조사에서 원활한 국제협력을 위해서도 필수적\\nn 그러나 법 집행에서 AI 도구의 효과적이고 책임 있는 활용을 위해 해결되어야 할 기술적 과제 및\\n다양한 윤리적·사회적 우려도 존재\\n∙ 일례로 관할권 간 데이터 수집과 보관 관행의 차이에 따른 데이터셋의 편향으로 인해 AI 산출물의\\n무결성(無缺性)이 손상될 수 있어 표준화된 데이터 수집 규약 필요\\n∙ 데이터 규모나 활용 사례의 복잡

In [158]:
import ast
def convert_to_list(example):
    contexts = ast.literal_eval(example['contexts'])
    return {'contexts':contexts}

In [159]:
test_dataset = test_dataset.map(convert_to_list)

Map: 100%|██████████| 10/10 [00:00<00:00, 303.53 examples/s]


In [161]:
test_dataset[0]['contexts']

['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간\nKEY Contents\nn 유로폴의 보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오\n분석 등에 활용되어 법 집행 업무를 대폭 개선할 수 있는 잠재력 보유\nn 그러나 AI 도입을 위해서는 기술적 과제 해결 및 다양한 윤리적·사회적 이슈 대응이\n필요하며, EU AI 법에 부합하도록 기존 AI 시스템에 대한 평가와 수정도 필요\n£유로폴, 법 집행에서 AI 기술의 윤리적이고 투명한 구현을 위한 고려사항 제시\nn EU 사법기관 유로폴(Europol)이 2024년 9월 24일 법 집행에서 효과적 범죄 퇴치를 위한 AI의\n활용 가능성을 탐색한 보고서를 발간\n∙ 보고서는 법 집행에서 AI 기술을 윤리적이고 투명하게 구현하기 위한 지침 역할을 하며, AI의 이점과\n과제를 함께 다룸으로써 법 집행에서 AI 사용 시 윤리적 고려 사항에 대한 인식 제고를 추구\nn 보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오 분석, 생체인식\n시스템 등에 활용되어 법 집행 업무를 대폭 개선할 수 있는 잠재력 보유\n∙ 법 집행기관은 AI 기반 데이터 분석을 활용해 범죄 활동에 대한 탐지와 대응 능력을 강화하고, AI\n도구로 구조화되지 않은 데이터를 신속히 분석해 비상 상황의 의사결정을 위한 통찰력 확보 가능\n∙ 기계번역과 같은 AI 기반 도구는 여러 국가가 참여하는 조사에서 원활한 국제협력을 위해서도 필수적\nn 그러나 법 집행에서 AI 도구의 효과적이고 책임 있는 활용을 위해 해결되어야 할 기술적 과제 및\n다양한 윤리적·사회적 우려도 존재\n∙ 일례로 관할권 간 데이터 수집과 보관 관행의 차이에 따른 데이터셋의 편향으로 인해 AI 산출물의\n무결성(無缺性)이 손상될 수 있어 표준화된 데이터 수집 규약 필요\n∙ 데이터 규모나 활용 사례의 복잡성과 관계없이 AI 도구를 효과적으로 사

In [162]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [163]:
loader = PyMuPDFLoader('data/SPRi AI Brief_11월호_산업동향_F.pdf')
docs = loader.load()

In [164]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
split_documents = text_splitter.split_documents(docs)

In [165]:
embeddings = OpenAIEmbeddings()

In [166]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [167]:
retriever = vectorstore.as_retriever()

In [168]:
prompt = PromptTemplate.from_template(
    """너는 주어진 질문에 대답하는 AI야. 다음 검색된 context를 사용해서 question에 대답해줘.
    답을 모르면, '알 수 없습니다.'라고 대답해.
    
    
    # Context : {context}
    # Question : {question}
    # Answer :
    """
)

In [174]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [175]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [176]:
batch_dataset = []
for question in test_dataset['question']:
    batch_dataset.append(question)

In [177]:
batch_dataset[:3]

['What specific aspects of data collection standardization, such as data format, quality control, or metadata standards, are required to effectively use AI tools, while ensuring ease of implementation and simplicity, without considering data size and utilization scenarios?',
 'What factors are contributing to the recent increase in popularity of the Mamba AI framework among developers?',
 'How does the MIT Industrial Performance Center assess the impact of automation technology on workers?']

In [178]:
answer = chain.batch(batch_dataset)
answer[:3]

['알 수 없습니다.',
 '최근 Mamba AI 프레임워크의 인기가 높아지는 이유는 다음과 같습니다:\n\n1. **효율적인 메모리 사용**: Mamba는 트랜스포머 모델의 핵심인 어텐션 메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여하여 메모리 사용을 최적화합니다. 이는 AI 에이전트의 효율성을 높이는 데 기여합니다.\n\n2. **빠른 추론 시간**: Mamba 아키텍처는 더 빠른 추론 시간을 지원하여 개발자들이 더 효율적으로 작업할 수 있도록 합니다.\n\n3. **오픈소스 개발자 사이의 관심**: 최근 미스트랄과 UAE의 AI 기업 팔콘이 Mamba 기반의 모델을 출시하면서 오픈소스 AI 개발자들 사이에서 Mamba의 인기가 높아지고 있습니다.\n\n이러한 요소들이 결합되어 Mamba AI 프레임워크의 인기를 증가시키고 있습니다.',
 'MIT 산업성과센터는 설문조사를 통해 근로자 관점에서 자동화 기술의 영향을 평가한 결과, 근로자들은 직장 내 안전, 임금, 업무 자율성 등에서 자동화를 긍정적으로 평가하고 있다고 밝혔습니다. 조사에 따르면, 복잡한 문제 해결이 필요한 작업을 수행하는 근로자 및 자신의 직무에 만족하는 근로자일수록 자동화의 영향에 긍정적인 것으로 나타났습니다. 또한, 자동화가 직장 내 안전에 긍정적인 영향을 미친다고 응답한 비율이 44.9%에 달하며, 임금과 업무 자율성에 대해서도 긍정적인 응답이 우세한 것으로 확인되었습니다. 그러나 국가별로 차이가 있으며, 미국 근로자들은 자동화가 임금 및 직업 안정성에 부정적이라는 응답이 더 많았습니다.']

In [179]:
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', answer)
else:
    test_dataset = test_dataset.add_column('answer', answer)

In [180]:
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 10
})

In [181]:
from ragas import evaluate
from ragas.metrics import answer_relevancy,faithfulness,context_recall,context_precision

In [182]:
result = evaluate(
    dataset = test_dataset,
    metrics = [
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision
    ]
)


# answer_relevancy : 모델이 문맥에서 중요한 정보를 잘 회상, 재현했는가
# faithfulness : 모델의 답변이 문맥에 기반해서 사실적인가(factual)
# context_recall : 전체 질문에 대해 모델의 답변이 질문과 얼마나 관련성이 있는가
# context_precision : 모델이 문맥에서 필요한 정보를 정확히 활용했는가

Evaluating:  18%|█▊        | 7/40 [02:11<05:17,  9.62s/it]Exception raised in Job[27]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-nS8RoafnzeRRZRKTRiJUJQQC on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[4]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-nS8RoafnzeRRZRKTRiJUJQQC on tokens per min (TPM): Limit 200000, Used 200000, Requested 2758. Please try again in 827ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Evaluating:  35%|███▌      | 14/40 [02:51<03:18,  7.63s/it]Exception raised in Job[39]: TimeoutError()
Exception raised in Job[36]: TimeoutError()
Evaluat

In [183]:
result

{'answer_relevancy': nan, 'faithfulness': nan, 'context_recall': nan, 'context_precision': nan}

In [184]:
result_df = result.to_pandas()
result_df

,question,contexts,answer,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What specific aspects of data collection stand...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n유로폴, 법 집...",알 수 없습니다.,The answer to given question is not present in...,NaN,NaN,NaN,NaN
1,What factors are contributing to the recent in...,"[SPRi AI Brief |\n2024-11월호\nAI21 CEO, AI 에이전트...",최근 Mamba AI 프레임워크의 인기가 높아지는 이유는 다음과 같습니다:\n\n1...,The recent increase in popularity of the Mamba...,NaN,NaN,NaN,NaN
2,How does the MIT Industrial Performance Center...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\nMIT 산업성과...,MIT 산업성과센터는 설문조사를 통해 근로자 관점에서 자동화 기술의 영향을 평가한 ...,The MIT Industrial Performance Center assesses...,NaN,NaN,NaN,NaN
3,What role does AI play in addressing global ch...,"[∙ (글로벌 도전과제 해결) 환경 문제, 경제 회복력, 사회복지 등 글로벌 도전 ...",AI plays a significant role in addressing glob...,AI helps in solving global challenges such as ...,NaN,NaN,NaN,NaN
4,How do Transformers boost AI21's model efficie...,"[SPRi AI Brief |\n2024-11월호\nAI21 CEO, AI 에이전트...",알 수 없습니다.,Transformers are used extensively in AI model ...,NaN,NaN,NaN,NaN
5,How can investment boost AI platforms in compa...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n가트너 예측, ...","기업의 AI 개발자 플랫폼에 대한 투자는 AI 엔지니어를 지원하고, AI 역량을 더...",Companies need to strengthen investment in AI ...,NaN,NaN,NaN,NaN
6,How might the EU AI law impact data use for cr...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n유로폴, 법 집...",EU AI 법은 공공장소에서 실시간 생체인식 식별과 같은 특정 애플리케이션의 사용을...,"The EU AI law, effective from August 2024, imp...",NaN,NaN,NaN,NaN
7,What limits Transformers in AI that require me...,"[SPRi AI Brief |\n2024-11월호\nAI21 CEO, AI 에이전트...",트랜스포머 아키텍처는 처리하는 컨텍스트가 길어질수록 속도가 느리고 연산 비용이 많이...,Transformers are limited in AI applications th...,NaN,NaN,NaN,NaN
8,"If LLMs don't become more reliable, how could ...",[메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사...,"If LLMs don't become more reliable, it could h...","If LLMs don't become more reliable, it could a...",NaN,NaN,NaN,NaN
9,How does AI safety shift with data privacy pri...,"[인간중심, 안전성\n비의도적 사용 대처 미발생\n개인정보 보호 프라이버시 보호 Ÿ...",알 수 없습니다.,The context does not provide specific informat...,NaN,NaN,NaN,NaN


In [185]:
result_df.loc[:, 'context_recall':'context_precision']

,context_recall,context_precision
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


|context_precision (문맥 정밀도)|faithfulness (신뢰성)|answer_relevancy (답변 적합성)|context_recall (문맥 재현율)|
|---|---|---|---|
|답변에 사용된 문맥이 정확했는지|답변이 문맥에 기반해 사실적(factual)인지|답변이 질문과 관련성이 있는지|답변이 문맥에서 얼마나 많은 정보를 활용했는지|
|1.0: 답변에 사용된 정보가 문맥에서 매우 정확하게 활용됨|1.0: 답변이 문맥과 매우 정확하게 일치|1.0: 답변이 질문과 매우 높은 관련성을 가짐|1.0: 답변이 문맥의 모든 중요한 정보를 잘 회상|
|0.0: 답변이 문맥과 무관하거나, 부정확한 정보를 활용|0.0: 답변이 문맥에 기반하지 않고, 사실과 다를 가능성이 큼|0.0: 답변이 질문과 관련이 없거나, 부정확|0.0: 답변이 문맥을 활용하지 못함|

In [186]:
pip install streamlit

  Using cached streamlit-1.40.2-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (